In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("udf").getOrCreate()

23/03/26 21:06:40 WARN Utils: Your hostname, Moon-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.2 instead (on interface en0)
23/03/26 21:06:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/26 21:06:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/26 21:06:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]

In [3]:
df = spark.createDataFrame(data = transactions, schema = schema)
# dataframe 생성 

In [4]:
# tempview
df.createOrReplaceTempView("transactions")

In [6]:
spark.sql("select * from transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [11]:
# UDF 정의하는 첫번째 방식 
from pyspark.sql.types import LongType
def squared(n):
    return n * n

# 정의한 함수를 spark session을 통해 sql에서 사용할 수 있도록 만들어줌 
spark.udf.register("squared", squared, LongType())

23/03/26 21:10:26 WARN SimpleFunctionRegistry: The function squared replaced a previously registered function.


<function __main__.squared(n)>

In [12]:
spark.sql("select price, squared(price) from transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [23]:
# UDF 정의하는 두번째 방식 
from pyspark.sql.functions import udf 

@udf("long")
def squared1(n):
    return n * n 
# 이거 안되네

In [24]:
spark.sql("select price, squared1(price) from transactions").show()

AnalysisException: Undefined function: squared1. This function is neither a built-in/temporary function, nor a persistent function that is qualified as spark_catalog.default.squared1.; line 1 pos 14

In [30]:
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    results = []
    i = 0 
    while n > 0:
        n,r = divmod(n, 10)
        # n을 10으로 나눴을 때 몫과 나머지 
        if r > 0: 
            results.append(nums[r-1] + units[i])
        i += 1 
    return "".join(reversed(results))

print(read_number(21520))

이만일천오백이십


In [37]:
spark.udf.register("read_number", read_number)
# udf 등록 

<function __main__.read_number(n)>

In [39]:
spark.sql("select price, read_number(price) from transactions").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [46]:
def get_weekday(date):
    import calendar
    return calendar.day_name[date.weekday()]
    # monday 등등으로 값 리턴 

spark.udf.register("get_weekday", get_weekday)

23/03/26 21:22:31 WARN SimpleFunctionRegistry: The function get_weekday replaced a previously registered function.


<function __main__.get_weekday(date)>

In [48]:
query = """
select 
    datetime,
    get_weekday(TO_DATE(datetime)) as dayOfWeek
FROM
    transactions
"""

spark.sql(query).show()

+-------------------+---------+
|           datetime|dayOfWeek|
+-------------------+---------+
|2021-11-07 13:20:00|   Sunday|
|2021-10-24 11:19:00|   Sunday|
|2021-07-25 11:12:40|   Sunday|
|2021-07-10 08:20:00| Saturday|
|2021-07-01 05:36:00| Thursday|
|2020-08-19 19:04:00|Wednesday|
+-------------------+---------+

